In [1]:
function countHeads(n::Int)
  c::Int = 0
  for i=1:n
    c += rand(Bool)
  end
  c
end

countHeads (generic function with 1 method)

In [2]:
@time countHeads(2*10^9)/(2*10^9)

  2.513408 seconds


0.500001624

In [3]:
using Distributed

In [4]:
nprocs()

1

In [5]:
addprocs(3)

3-element Vector{Int64}:
 2
 3
 4

In [8]:
nprocs()

2

In [15]:
@everywhere function countHeads(n::Int)
  c::Int = 0
  for i=1:n
    c += rand(Bool)
  end
  c
end

In [10]:
a= @spawn countHeads(10^9)
b= @spawn countHeads(10^9)

Future(2, 1, 6, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (4525748928, 4525749264, 0)), nothing)

In [11]:
fetch(a)+fetch(b)

1000021582

In [12]:
a= @spawn countHeads(10^9)
b= @spawn countHeads(10^9)
@time fetch(a)+fetch(b)

  2.469003 seconds (204 allocations: 11.453 KiB, 1 lock conflict)


999962992

In [16]:
a= @spawn countHeads(5*10^8)
b= @spawn countHeads(5*10^8)
c= @spawn countHeads(5*10^8)
d= @spawn countHeads(5*10^8)
@time fetch(a)+fetch(b)+fetch(c)+fetch(d)

  1.719634 seconds (371 allocations: 22.766 KiB)


1000012099

In [25]:
@time let
  nheads = @distributed (+) for i = 1:2*10^10
    Int(rand(Bool))
  end
end

 12.788984 seconds (15.46 k allocations: 786.367 KiB, 0.10% compilation time)


9999929928

In [24]:
nprocs()

3

In [23]:
rmprocs(1)

┌ Warning: rmprocs: process 1 not removed
└ @ Distributed /Users/pstaab/.julia/juliaup/julia-1.11.0-rc1+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Distributed/src/cluster.jl:1049


Task (done) @0x000000010c6595f0

In [6]:
procs()

4-element Vector{Int64}:
 1
 2
 3
 4

In [12]:
rmprocs(2,3,4)

Task (done) @0x000000015d3e0c90

In [7]:
addprocs()

4-element Vector{Int64}:
 5
 6
 7
 8

In [13]:
procs()

1-element Vector{Int64}:
 1

In [18]:
@time let
  nheads = @distributed (+) for i = 1:2*10^10
    Int(rand(Bool))
  end
end

  5.423481 seconds (16.58 k allocations: 860.430 KiB, 0.20% compilation time)


10000028560

In [17]:
addprocs()

4-element Vector{Int64}:
 12
 13
 14
 15

In [1]:
include("PlayingCards.jl")
using .PlayingCards, Random, Distributed

In [2]:
function countHands(trials::Int,f::Function)
  local deck=map(Card,1:52)
  local num_hands=0
  for i=1:trials
      shuffle!(deck)
      h = Hand(deck[1:5])
      if(f(h))
          num_hands+=1
      end
  end
  num_hands
end

countHands (generic function with 1 method)

In [3]:
@time countHands(10_000_000,isFullHouse)

  4.305718 seconds (60.00 M allocations: 3.129 GiB, 4.61% gc time)


14430

In [1]:
using Distributed
@everywhere include("PlayingCards.jl")
@everywhere using .PlayingCards, Random

In [2]:
@everywhere function paraCountHands(trials::Integer,f::Function)
  local deck=map(Card,1:52)
  function checkHand(f::Function) ## shuffle the deck then check the hand.
    shuffle!(deck)
    f(Hand(deck[1:5]))
  end
  @distributed (+) for i = 1:trials
    Int(checkHand(f))
  end
end

In [3]:
addprocs(3)

3-element Vector{Int64}:
 2
 3
 4

In [4]:
@time fh = paraCountHands(10_000_000,isFullHouse)

TaskFailedException: TaskFailedException

    nested task error: On worker 2:
    UndefVarError: `PlayingCards` not defined in `Main`
    Suggestion: add an appropriate import or assignment. This global was declared but not assigned.
    Stacktrace:
      [1] deserialize_module
        @ ~/.julia/juliaup/julia-1.11.0-rc1+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Serialization/src/Serialization.jl:1027
      [2] handle_deserialize
        @ ~/.julia/juliaup/julia-1.11.0-rc1+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Serialization/src/Serialization.jl:926
      [3] deserialize
        @ ~/.julia/juliaup/julia-1.11.0-rc1+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Serialization/src/Serialization.jl:844
      [4] deserialize_datatype
        @ ~/.julia/juliaup/julia-1.11.0-rc1+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Serialization/src/Serialization.jl:1470
      [5] handle_deserialize
        @ ~/.julia/juliaup/julia-1.11.0-rc1+0.aarch64.apple.darwin14/share/j